# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
%cd "/content/gdrive/My Drive/PerfPred/Experiment"

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/PerfPred/Experiment/src')

# Trial Vars

## Expr 1A: Size
- $D_1$ (`[TRAIN1_SIZE]`)
- $D_2$ (`[TRAIN2_SIZE]`)
- $D_1, D_2$ (`[TRAIN1_SIZE, TRAIN2_SIZE]`)

## Expr 1B: Domain Relatedness
- $j_1$ (`[TRAIN1_JSD]`)
- $j_2$ (`[TRAIN2_JSD]`)
- $j_1, j_2$ (`[TRAIN1_JSD, TRAIN2_JSD]`)

## Expr 1C: Language Relatedness (Dataset Independent)
- $d_\text{fea}$ (`[FEA_DIST]`)
- $d_\text{inv}$ (`[INV_DIST]`)
- $d_\text{pho}$ (`[PHO_DIST]`)
- $d_\text{syn}$ (`[SYN_DIST]`)
- $d_\text{gen}$ (`[GEN_DIST]`)
- $d_\text{geo}$ (`[GEO_DIST]`)
- $d_\text{inv}, d_\text{pho}$ (`[INV_DIST, PHO_DIST]`)
- $d_\text{inv}, d_\text{syn}$ (`[INV_DIST, SYN_DIST]`)
- $d_\text{pho}, d_\text{syn}$ (`[PHO_DIST, SYN_DIST]`)
- $d_\text{gen}, d_\text{geo}$ (`[GEN_DIST, GEO_DIST]`)
- $d_\text{inv}, d_\text{pho}, d_\text{syn}$ (`[INV_DIST, PHO_DIST, SYN_DIST]`)
- $d_\text{fea}, d_\text{gen}, d_\text{geo}$ (`[FEA_DIST, GEN_DIST, GEO_DIST]`)
- $d_\text{fea}, d_\text{inv}, d_\text{pho}, d_\text{syn}$ (`[FEA_DIST, INV_DIST, PHO_DIST, SYN_DIST]`)
- $d_\text{inv}, d_\text{pho}, d_\text{syn}, d_\text{gen}, d_\text{geo}$ (`[INV_DIST, PHO_DIST, SYN_DIST, GEN_DIST, GEO_DIST]`)
- $d_\text{fea}, d_\text{inv}, d_\text{pho}, d_\text{syn}, d_\text{gen}, d_\text{geo}$ (`[FEA_DIST, INV_DIST, PHO_DIST, SYN_DIST, GEN_DIST, GEO_DIST]`)


# Trial Functions

## General

### Linear
$$\text{linear}(x_1, \dots, x_n) = c_0 + \sum_{i=1}^n c_ix_i$$


### Polynomial
$$\text{polynomial}(x_1, \dots, x_n) = c_0 + \sum_{i=1}^n \sum_{j=1}^k c_{i, j} x_i^j$$
*Note:* You can look at $c$ as a constast $c_0$ and a $n \times k$ matrix $\{c_{i, j}\}$, but it's actually stored as a vector of length $1 + n \times k$ with $c_{i, j}$ stored at index $n(i - 1) + j$.


### Exponential
$$\text{exponential}(x_1, \dots, x_n) = c_0\exp\left(\sum_{i=1}^n c_ix_i\right)$$


### Logarithmic
$$\text{logarithmic}(x_1, \dots, x_n) = c_0 + \sum_{i=1}^n c_i\log(x_i)$$


### Power
$$\text{power}(x_1, \dots, x_n) = c_0 \sum_{i=1}^n x_i^{c_i}$$


### Multiplicative
$$\text{multiplicative}(x_1, \dots, x_n) = c_0 \prod_{i=1}^n x_i^{c_i}$$


### Hybrid Multiplicative
$$\text{hybrid-multiplicative}(x_1, \dots, x_n) = c_0 + \prod_{i=1}^n x_i^{c_i}$$


### Arithmetic Mean Linear
$$\text{arithmetic-mean-linear}(x_1, \dots, x_n) = c_0 + c_1\frac{\sum_{i=1}^n x_i}{n}$$


### Geometric Mean Linear
$$\text{geometric-mean-linear}(x_1, \dots, x_n) = c_0 + c_1\left(\prod_{i=1}^n x_i\right)^{\frac1n}$$


### Harmonic Mean Linear
$$\text{harmonic-mean-linear}(x_1, \dots, x_n) = c_0 + c_1\frac{n}{\sum_{i=1}^n \frac{1}{x_i}}$$


## Specific

### Scaling Law
$$\text{scaling_law} (x) = c_0 \left(x^{-1} + c_1 \right)^{c_2}$$
Idea from [Bansal's paper](https://arxiv.org/pdf/2202.01994.pdf), p.3.

### Anthony's Law
$$\text{sp-BLEU}(x_1,x_2) = c_0 (x_1x_2)^{-c_1} + c_2 x_2 ^ {-c_3} + c_4$$
See curve-fitting -> equation in [Anthony's work](https://colab.research.google.com/drive/1Rx6sExWQ9RsNQeoHwBSmzIP2D-XvtMRy#scrollTo=aC47KqM31nLO).


### Linear with Difference
$$\text{sp-BLEU}(x_1, x_2) = c_0 + c_1 x_1 + c_2 x_2 + c_3 |x_1 - x_2|$$

# TODO (Hasti)

### Trial 4: Simple Decision D1D2
$$
\text{sp-BLEU}(D_1, D_2) = \begin{cases}
  c_1 D_1 + c_2 D_2 + C &, D_1 > 10k \\
  c_2 D_2 + C &, \text{otherwise}
\end{cases}
$$


In [ ]:
def simple_decision_size(c,x):
  """ See above
  c: Array with dim 3, corresponding to c1, c2, and C
  x: Array of dim (n,2)
  y: Array with dim n
  """
  if np.all(x[:, 0] > 10):
    return c[0] * x[:, 0] + c[1] * x[:, 1] + c[2];
  return c[1] * x[:, 1] + c[2]

### Trial 2: Stepwise Regression from Linear Single


```
candidate_factors = [geo, gen, inv, syn, pho]
candidate_factors.sort() # Ascending based on average RMSE of single var linear

selected_factors = []
MAX_FACTORS = 5

# Start with linear single with lowest RMSE
current_model = linear_reg(candidate_factors[0])
best_rmse = rmse(linear_reg(candidate_factors[0]))
candidate_factors.pop_front()

# Perform stepwise regression
while len(selected_factors) < MAX_FACTORS:

  best_factor = None

  # Iterate over all remaining factors
  for factor in candidate_factors:

    # Add the candidate factor to current
    subset_factors = selected_factors
    subset_factors.append(factor)
    updated_model = linear_reg(subset_factors)
    rmse = rmse(updated_model)

    if rmse < best_rmse:
      best_factor = factor
      best_rmse = rmse

  if best_factor = None:
    break

  selected_factors.append(best_factor)
  candidate_factors.remove(best_factor)

  current_model = add_factor(current_model, best_factor)

final_model = current_model # Do whatever analysis with this
final_rmse = best_rmse

```




### Trial 3: Reverse Stepwise Regression from Linear Single


```
candidate_factors = [geo, gen, inv, syn, pho]
selected_factors = candidate_factors.copy()
current_model = linear_reg(candidate_factors)
best_rmse = rmse(simple_linear(candidate_factors))

for factor in candidate_factors:

  # Temporarily remove a factor
  subset_factors = selected_factors.copy()
  subset_factors.remove(factor)
  updated_model = linear_reg(subset_factors)
  current_rmse = rmse(updated_model)

  if cur_rmse < best_rmse:
    best_remse = current_rmse
    selected_factors = subset_factors


final_model = current_model # Do whatever analysis with this
final_rmse = best_rmse


```



# Code

In [ ]:
import experiment.run as R
import experiment.plot as P
from modeling.trial import Trial as Tr
from slicing.variable import Variable as V

In [ ]:
R.run_on_trials(Tr.read_or_fit, vars=[[V.FEA_DIST], [V.INV_DIST], [V.PHO_DIST], [V.SYN_DIST], [V.GEN_DIST], [V.GEO_DIST]])

In [ ]:
R.run_on_experiments(P.plot_compact, vars=[[V.FEA_DIST], [V.INV_DIST], [V.PHO_DIST], [V.SYN_DIST], [V.GEN_DIST], [V.GEO_DIST]])